In [1]:
from main.lda.model import LdaModelGenerator, LdaGeneratorConfig

corpus = "../../output/ds/pre_processed.80k.csv"

config = LdaGeneratorConfig(corpus_file_path=corpus)
generator = LdaModelGenerator(config)

In [2]:
base_model = generator.generate_training_model()

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

In [6]:
import pandas as pd
from main.lda.dataset import LdaDataset
from gensim.models import CoherenceModel

topn = 25
coh_model = CoherenceModel(base_model, corpus=LdaDataset(pd.read_csv(config.corpus_file_path)).dataset, coherence='u_mass', topn=topn)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

In [7]:
coh_model.get_coherence_per_topic()

[-4.734980166157313,
 -4.772950539925173,
 -6.465469420055531,
 -4.9681191893395,
 -4.234377075568865,
 -4.0297838392725325,
 -6.416305637311228,
 -5.470809434039697,
 -6.112251091414597,
 -6.356242827866072,
 -5.119788831658615,
 -3.812288790483861,
 -6.910567220917148,
 -6.625797016387571]

In [8]:
coh_model.get_coherence()

-5.430695077171264

In [ ]:
# Now the model has high coherence which is good but are the extracted aspects good?
# Probably not.